<h1><center>Stock Market Trading Bot</center></h1>

# Table of contents
* [Introduction: Business Problem](#introduction)
* [OHLC Data]() 
    * [Loading Data](#OHLC_Load)
    * [Plotting Data](#Plt)
    * [Descriptive Statistics](#desc)
    * [Inferential Statistics](#Inf)
    * [Trading FrameWork](#Tf)
    * [Trade Log](#Tl)
    * [Unit Testing](#Ut)
* [Per Trade Data]()
    * [Loading Data](#pt)
    * [Combining Date Time](#dt)

# Importing libraries

In [ ]:
! pip install plotly

In [1]:
import csv
import time
import math
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
import matplotlib as mpl
from datetime import datetime
import matplotlib.pyplot as plt

# Data Source

[Data Source](https://www.kaggle.com/ghazanfarali/ksedataset?select=UBL.csv) 
[Data Source](https://www.kaggle.com/arsalanjaved/pakistan-stock-exchange)

# Loading data <a name='OHLC_Load'></a>

In [ ]:
UBL=pd.read_csv('Data/UBL.csv',index_col='Date',parse_dates=True)
HBL=pd.read_csv('Data/HBL.csv',index_col='Date',parse_dates=True)
NBP=pd.read_csv('Data/NBP.csv',index_col='Date',parse_dates=True)
MEBL=pd.read_csv('Data/MEBL.csv',index_col='Date',parse_dates=True)
UBL.head()

# Plotting Data <a name='Plt'></a>

In [ ]:
plt.figure(figsize=(12,8))
sns.lineplot(data=UBL.Close,label='UBL')
sns.lineplot(data=HBL.Close,label='HBL')
sns.lineplot(data=MEBL.Close,label='MEBL')
sns.lineplot(data=NBP.Close,label='NBP')
plt.xlabel('Date',fontsize=15)
plt.ylabel('Closing Price',fontsize=15)
plt.title('Trend of Share Price of different KSE-30 Companies',fontsize=15)

## Dropping redundant feature

In [ ]:
UBL.drop('Symbol',axis=1,inplace=True)
MEBL.drop('Symbol',axis=1,inplace=True)
NBP.drop('Symbol',axis=1,inplace=True)
HBL.drop('Symbol',axis=1,inplace=True)

## Changing column names in order to facilitate merge

In [ ]:
UBL.columns=['UBL_Open','UBL_High','UBL_Low','UBL_Close','UBL_Volume']
MEBL.columns=['MEBL_Open','MEBL_High','MEBL_Low','MEBL_Close','MEBL_Volume']
HBL.columns=['HBL_Open','HBL_High','HBL_Low','HBL_Close','HBL_Volume']


## Merging Data Frame

In [ ]:
data=pd.concat([UBL,MEBL,HBL],axis=1)
data.info()

## Reversing Data Frame

In [ ]:
data1=data.iloc[::-1]
data1=data1.reset_index()
data1=data1.iloc[::-1]
data1=data1.set_index('Date')

In [ ]:
x=data1

In [ ]:
x.head()

## Dropping Missing Values

In [ ]:
data1.dropna(inplace=True)

In [ ]:
data1.isnull().sum().sum()

# Descriptive Statistics <a name='desc'></a>

In [ ]:
data2=data1.loc[:,['UBL_Close','MEBL_Close','HBL_Close']]

In [ ]:
plt.figure(figsize=(10,7))
sns.heatmap(data=data2.corr())

## Box plot

In [ ]:
plt.figure(figsize=(14,10))
sns.boxplot(data=data2)
plt.xlabel('Date',fontsize=15)
plt.ylabel('Closing Price',fontsize=15)
plt.title('Trend of Share Price of different KSE-30 Companies',fontsize=15)

# Inferential Statistics <a name='Inf'></a>

## Pearson Correlation

In [ ]:
pearson_coeff,p_val=stats.pearsonr(data2['UBL_Close'],data2['HBL_Close'])
pearson_coeff,p_val

**_Conclusion:_**

Since the p-value is <
0.001, the correlation between UBL Stock Closing Price and HBL Stock Closing Price is statistically significant, and the linear relationship is quite strong (~0.834).

In [ ]:
pearson_coeff,p_val=stats.pearsonr(data2['UBL_Close'],data2['MEBL_Close'])
pearson_coeff,p_val

**_Conclusion:_**

Since the p-value is <
0.001, the correlation between UBL Stock Closing Price and HBL Stock Closing Price is statistically significant, and the linear relationship is quite strong (~0.738).

In [ ]:
pearson_coeff,p_val=stats.pearsonr(data2['MEBL_Close'],data2['HBL_Close'])
pearson_coeff,p_val

# Trading Frame Work <a name='Tf'></a>

In [ ]:
def trader(X,nS1,nS3,share_price1,share_price2):
    Amount_Gained=((nS3*share_price2)*.1)
    Brocker_fee=((nS3*share_price2)*.1)*.3*.01
    nS3=nS3-(nS3*.1)
    X=X+Amount_Gained-Brocker_fee
    Shares_bought=(.997*Amount_Gained)/share_price1
    nS1=nS1+Shares_bought
    X=X-(.997*Amount_Gained)
    return X,nS1,nS3,Amount_Gained,-(Shares_bought*share_price1),Brocker_fee

## Refactored Code

In [ ]:
def stock1(X,nS1,nS2,nS3,pS1,pS2,pS3,share_prices,Prediction,TN,low_thres,high_thres):
    Trade_Occurred,Brocker_fee,Share1,Share2,Share3,temp1,temp2,temp3=False,0,0,0,0,0,0,0
    if (pS3/TN)>low_thres and (pS2/TN)>low_thres:
            X,nS1,nS3,temp1,temp2,temp3=trader(X,nS1,nS3,share_price[0],share_price[2])
            Share3+=temp1
            Share1+=temp2
            Brocker_fee+=temp3
            X,nS1,nS2,temp1,temp2,temp3=trader(X,nS1,nS2,share_price[0],share_price[1])
            Share2+=temp1
            Share1+=temp2
            Brocker_fee+=temp3
            Trade_Occurred=True
    elif (model2_pred>model3_pred) and (pS3/TN)>low_thres:
            X,nS1,nS3,temp1,temp2,temp3=trader(X,nS1,nS3,share_price[0],share_price[2])
            Share3+=temp1
            Share1+=temp2
            Brocker_fee+=temp3
            Trade_Occurred=True
    elif (model3_pred>model2_pred) and (pS2/TN)>low_thres:
            X,nS1,nS2,temp1,temp2,temp3=trader(X,nS1,nS2,share_price[0],share_price[1])
            Share2+=temp1
            Share1+=temp2
            Brocker_fee+=temp3
            Trade_Occurred=True
    if model2_pred<0 and (pS2/TN)>low_thres:
            X,nS1,nS2,temp1,temp2,temp3=trader(X,nS1,nS2,share_price[0],share_price[1])
            Share2+=temp1
            Share1+=temp2
            Brocker_fee+=temp3
            Trade_Occurred=True
    if model3_pred<0 and (pS3/TN)>low_thres:
            X,nS1,nS3,temp1,temp2,temp3=trader(X,nS1,nS3,share_price[0],share_price[2])   
            Share3+=temp1
            Share1+=temp2
            Brocker_fee+=temp3
            Trade_Occurred=True
    return X,nS1,nS2,nS3,Trade_Occurred,Share1,Share2,Share3,Brocker_fee

In [ ]:
def stock2(X,nS1,nS2,nS3,pS1,pS2,pS3,share_prices,Prediction,TN,low_thres,high_thres):
    Trade_Occurred,Brocker_fee,Share1,Share2,Share3,temp1,temp2,temp3=False,0,0,0,0,0,0,0
    if (pS3/TN)>low_thres and (pS2/TN)>low_thres:
            X,nS2,nS3,temp1,temp2,temp3=trader(X,nS2,nS3,share_price[1],share_price[2])
            Share3+=temp1
            Share2+=temp2
            Brocker_fee+=temp3
            X,nS2,nS1,temp1,temp2,temp3=trader(X,nS2,nS1,share_price[1],share_price[0])
            Share1+=temp1
            Share2+=temp2
            Brocker_fee+=temp3
            Trade_Occurred=True
    elif (model1_pred>model3_pred) and (pS3/TN)>low_thres:
            X,nS2,nS3,temp1,temp2,temp3=trader(X,nS2,nS3,share_price[1],share_price[2])
            Share3+=temp1
            Share2+=temp2
            Brocker_fee+=temp3
            Trade_Occurred=True
    elif (model3_pred>model2_pred) and (pS1/TN)>low_thres:
            X,nS2,nS1,temp1,temp2,temp3=trader(X,nS2,nS1,share_price[1],share_price[0])
            Share1+=temp1
            Share2+=temp2
            Brocker_fee+=temp3
            Trade_Occurred=True
    if model1_pred<0 and (pS1/TN)>low_thres:
            X,nS2,nS1,temp1,temp2,temp3=trader(X,nS2,nS1,share_price[1],share_price[0])
            Share1+=temp1
            Share2+=temp2
            Brocker_fee+=temp3
            Trade_Occurred=True
    if model3_pred<0 and (pS3/TN)>low_thres:
            X,nS2,nS3,temp1,temp2,temp3=trader(X,nS2,nS3,share_price[1],share_price[2])
            Share3+=temp1
            Share2+=temp2
            Brocker_fee+=temp3
            Trade_Occurred=True
    return X,nS1,nS2,nS3,Trade_Occurred,Share1,Share2,Share3,Brocker_fee    

In [ ]:
def stock3(X,nS1,nS2,nS3,pS1,pS2,pS3,share_prices,Prediction,TN,low_thres,high_thres):
    Trade_Occurred,Brocker_fee,Share1,Share2,Share3,temp1,temp2,temp3=False,0,0,0,0,0,0,0
    if (pS3/TN)>low_thres and (pS2/TN)>low_thres:
        X,nS3,nS1,temp1,temp2,temp3=trader(X,nS3,nS1,share_price[2],share_price[0])
        Share1+=temp1
        Share3+=temp2
        Brocker_fee+=temp3
        X,nS3,nS2,temp1,temp2,temp3=trader(X,nS3,nS2,share_price[2],share_price[1])
        Share2+=temp1
        Share3+=temp2
        Brocker_fee+=temp3
        Trade_Occurred=True
    elif (model2_pred>model1_pred) and (pS1/TN)>low_thres:
        X,nS3,nS1,temp1,temp2,temp3=trader(X,nS3,nS1,share_price[2],share_price[0])
        Share1+=temp1
        Share3+=temp2
        Brocker_fee+=temp3
        Trade_Occurred=True
    elif (model1_pred>model2_pred) and (pS2/TN)>low_thres:
        X,nS3,nS2,temp1,temp2,temp3=trader(X,nS3,nS2,share_price[2],share_price[1])
        Share2+=temp1
        Share3+=temp2
        Brocker_fee+=temp3
        Trade_Occurred=True
    if model2_pred<0 and (pS2/TN)>low_thres:
        X,nS3,nS2,temp1,temp2,temp3=trader(X,nS3,nS2,share_price[2],share_price[1])
        Share2+=temp1
        Share3+=temp2
        Brocker_fee+=temp3
        Trade_Occurred=True
    if model1_pred<0 and (pS1/TN)>low_thres:
        X,nS3,nS1,temp1,temp2,temp3=trader(X,nS3,nS1,share_price[2],share_price[0])
        Share1+=temp1
        Share3+=temp2
        Brocker_fee+=temp3
        Trade_Occurred=True
    return X,nS1,nS2,nS3,Trade_Occurred,Share1,Share2,Share3,Brocker_fee

In [ ]:
def shares_trading(X,nS1,nS2,nS3,share_prices,Prediction,TN,low_thres,high_thres):
    model1_pred,model2_pred,model3_pred=Prediction[0],Prediction[1],Prediction[2]
    pS1,pS2,pS3=(nS1*share_prices[0]),(nS2*share_prices[1]),(nS3*share_prices[2])
    Trade_Occurred,Brocker_fee,Share1,Share2,Share3=False,0,0,0,0
    if (model1_pred>model2_pred and model1_pred>model3_pred) and (pS1/TN)<high_thres:
        X,nS1,nS2,nS3,Trade_Occurred,Share1,Share2,Share3,Brocker_fee=stock1(X,nS1,nS2,nS3,pS1,pS2,pS3,
                                                                      share_prices,Prediction,TN,
                                                                      low_thres,high_thres)
        return X,nS1,nS2,nS3,Trade_Occurred,Share1,Share2,Share3,Brocker_fee
    
    elif (model2_pred>model1_pred and model2_pred>model3_pred) and (pS2/TN)<high_thres:
        X,nS1,nS2,nS3,Trade_Occurred,Share1,Share2,Share3,Brocker_fee=stock2(X,nS1,nS2,nS3,pS1,pS2,pS3,
                                                                      share_prices,Prediction,TN,
                                                                      low_thres,high_thres)
        return X,nS1,nS2,nS3,Trade_Occurred,Share1,Share2,Share3,Brocker_fee
    elif (model3_pred>model1_pred and model3_pred>model2_pred) and (pS3/TN)<high_thres:
        X,nS1,nS2,nS3,Trade_Occurred,Share1,Share2,Share3,Brocker_fee=stock3(X,nS1,nS2,nS3,pS1,pS2,pS3,
                                                                      share_prices,Prediction,TN,
                                                                      low_thres,high_thres)
        return X,nS1,nS2,nS3,Trade_Occurred,Share1,Share2,Share3,Brocker_fee
    else:
        Amount_Gained,Brocker_Fee=0,0
        return X,nS1,nS2,nS3,Trade_Occurred,Share1,Share2,Share3,Brocker_fee


## Front End of Trading Framework

In [ ]:
%matplotlib notebook
plt.rcParams['animation.html']='jshtml'

In [ ]:
data1.index=data1.index.astype('str')
fig=plt.figure(figsize=(10,6))
ax=fig.add_subplot(111)
i=0
x,y=[],[]
x1,y1=[],[]
x2,y2=[],[]
while True:
    x.append(data1.index[i])
    y.append(data1.UBL_Close[i])
    x1.append(data1.index[i])
    y1.append(data1.HBL_Close[i])
    x2.append(data1.index[i])
    y2.append(data1.MEBL_Close[i])
    ax.plot(x,y,color='blue')
    ax.plot(x1,y1,color='green')
    ax.plot(x2,y2,color='purple')
    plt.xticks(rotation=90)
    ax.legend(['UBL','HBL','MEBL'],loc='upper left')
    ax.set_xlabel('Date')
    ax.set_ylabel('Stock_Price')
    ax.set_title('Variations of Stock Prices over time')
    plt.show()
    fig.canvas.draw()
    time.sleep(1)
    i+=1

## Clonning DataFrame 

In [ ]:
data3=data1.reset_index()
data3=data3.loc[0:,:]
data3.reset_index(inplace=True)
data3.drop('index',inplace=True,axis=1)
data3

## Creating Data Frame to Record Files into Excel Sheet

In [ ]:
Data_Acquired=pd.DataFrame({'Time_Stamp':['2009-06-18','2009-06-18'],'UBL_Shares':[0,162],'HBL_Shares':[0,92],'MEBL_Shares':[0,986],'Trade_Ocurred':[False,True],
                            'UBL_Pred':[0,0],'HBL_Pred':[0,0],'MEBL_Pred':[0,0],'UBL_portfolio':[0,.33],
                            'HBL_portfolio':[0,.33],'MEBL_portfolio':[0,.33],'Balance':[100000,109],'Brocker_Fee':[0,(99000*.3*.01*3)]
                            ,'Share1_NetChange':[0,-33000],'Share2_NetChange':[0,-33000],'Share3_NetChange':[0,-33000],'TN':[100000,99109]})
Data_Acquired['Time_Stamp']=pd.to_datetime(Data_Acquired['Time_Stamp'],format='%Y-%m-%d')
Data_Acquired.to_csv('Trade_log.csv',index=None)

In [ ]:
Data_Acquired

# Unit Testing <a name='Ut'></a>

In [ ]:
X=100000
nS1=math.floor((X*.33)/data3.loc[0,'UBL_Close'])
nS2=math.floor((X*.33)/data3.loc[0,'HBL_Close'])
nS3=math.floor((X*.33)/data3.loc[0,'MEBL_Close'])
X-=X*.99+(891)
X,nS1,nS2,nS3

In [ ]:
X=100000
nS1=(X*.33)/data3.loc[0,'UBL_Close']
nS2=(X*.33)/data3.loc[0,'HBL_Close']
nS3=(X*.33)/data3.loc[0,'MEBL_Close']

X-=X*.99+(891)
low_thres=0.3
high_thres=0.4
for index in range(data3.shape[0]-1):
    total_shares=nS1+nS2+nS3
    model1_pred=data3.loc[index+1,'UBL_Close']-data3.loc[index,'UBL_Close']
    model2_pred=data3.loc[index+1,'HBL_Close']-data3.loc[index,'HBL_Close']
    model3_pred=data3.loc[index+1,'MEBL_Close']-data3.loc[index,'MEBL_Close']
    prediction=[model1_pred,model2_pred,model3_pred]
    
    share_price=[data3.loc[index,'UBL_Close'],data3.loc[index,'HBL_Close'],data3.loc[index,'MEBL_Close']]
    TN=(nS1*data3.loc[index,'UBL_Close'])+(nS2*data3.loc[index,'HBL_Close'])+(nS3*data3.loc[index,'MEBL_Close'])
    
    X,nS1,nS2,nS3,Trade_Occurred,Share1,Share2,Share3,Brocker_fee=shares_trading(X,nS1,nS2,nS3,
                                                                          share_price,prediction,TN,low_thres,
                                                                          high_thres)
    
    pS1,pS2,pS3=(nS1*data3.loc[index,'UBL_Close']),(nS2*data3.loc[index,'HBL_Close']),(nS3*data3.loc[index,'MEBL_Close'])
    TN=X+(nS1*data3.loc[index,'UBL_Close'])+(nS2*data3.loc[index,'HBL_Close'])+(nS3*data3.loc[index,'MEBL_Close'])


    with open('Trade_log.csv','a',newline='') as raw:
        file_writer=csv.writer(raw)
        file_writer.writerow([data3.loc[index,'Date'],math.floor(nS1),math.floor(nS2),math.floor(nS3),Trade_Occurred,model1_pred,model2_pred,model3_pred,
                              pS1/TN,pS2/TN,pS3/TN,X,Brocker_fee,Share1,Share2,Share3,TN])

# Trade Log <a name='Tl'></a>

In [ ]:
d=pd.read_csv('Trade_log.csv')
d

In [ ]:
TN

In [ ]:
pS1/100000,pS2/100000,pS3/100000

In [ ]:
plt.figure(figsize=(10,6))
sns.lineplot(data=d.iloc[:2481,1:4])
plt.xticks(rotation=90)
plt.xlabel('Row in Dataset')
plt.ylabel('Number Of Stocks')
plt.title('Variations in Number of Stocks')
plt.legend(['UBL_Stocks','HBL_Stocks','MEBL_Stocks'],loc='upper left')

In [ ]:
plt.figure(figsize=(10,6))
sns.lineplot(data=d.iloc[:,16])
plt.xticks(rotation=90)
plt.xlabel('Row in Dataset')
plt.ylabel('Net Worth')
plt.title('Variation in Networth')
plt.text(x=1300,y=(.2*(10^7)),s='Boom')
plt.legend(['NetWorth Variation'],loc='upper left')

## Why 2013 Boom?

### IMF Support relief Programme

![IMF Relief Programme](IMF_PRESS_CUT.png)

### CPEC

![Press Release](CPEC.png)

In [ ]:
plt.figure(figsize=(10,6))
sns.lineplot(data=d.iloc[:201,8:11])
plt.xticks(rotation=90)
plt.xlabel('Row in Dataset')
plt.ylabel('Percentage of Each Stocks')
plt.title('Variation of Stocks in Portfolio overtime')
plt.legend(['UBL_Stocks','HBL_Stocks','MEBL_Stocks'],loc='upper left')

## Investment Portfolio

In [ ]:
total_shares=nS1+nS2+nS3
X,nS1/total_shares,nS2/total_shares,nS3/total_shares

# Per Trade Data <a name='pt'></a>

In [ ]:
Names=['ATRL','bahl','EPCL','FCCL','LUCK','MCB','MLCF','NETSOL','NML',
      'OGDC','PPL','PSO','SEARL','TRG']
i,dataframes=0,[]
for name in Names:
    dataframes.append(pd.read_csv('../per_trade_data/'+name+'.txt',delimiter='|',index_col='TRADE_ENTRY_DATE',
                        parse_dates=True))
    dataframes[i].reset_index(inplace=True)
    i+=1

In [ ]:
dataframes

# Improving DateTime Format <a name='dt'></a>

In [ ]:
def Format(t):
    return t[:1]+':'+t[1:3]+':'+t[3:5]+'.'+t[5:] if len(t)==9 else t[:2]+':'+t[2:4]+':'+t[4:6]+'.'+t[6:]


In [ ]:
def Combined(date,time):
    return date[:11]+Format(str(time))

In [ ]:
def Formated(row):
    return Combined(str(row.TRADE_ENTRY_DATE),str(row.KATS_TIME))


In [ ]:
Names=['ATRL','bahl','EPCL','FCCL','LUCK','MCB','MLCF','NETSOL','NML',
      'OGDC','PPL','PSO','SEARL','TRG']
i,dataframes=0,[]
for i in range(14):
    dataframes[i]['Date_Time']=dataframes[i].apply(Formated,axis='columns')
    dataframes[i]['Date_Time']=pd.to_datetime(dataframes[i]['Date_Time'], infer_datetime_format=True)
    dataframes[i].to_csv('../Modified_Data/Modified_'+Names[i]+'.csv',index='Date_Time')

In [2]:
df=pd.read_csv('../Modified_Data/Modified_ATRL.csv',parse_dates=True,index_col='Date_Time',infer_datetime_format=True)
df.drop(['Unnamed: 0','KATS_TIME'],axis=1,inplace=True)
df.columns=['TRADE_ENTRY_DATE','ATRL','TRADE_QTY','TRADE_RATE']

In [3]:
df.reset_index(inplace=True)

In [11]:
df_day1=df[(df.Date_Time.dt.year==2009) & (df.Date_Time.dt.day==1) & (df.Date_Time.dt.month==4)]

In [21]:
df1=df_day1[:]
df2=df1.groupby([df1.TRADE_ENTRY_DATE,df1.Date_Time.dt.hour,df1.Date_Time.dt.minute,df1.ATRL]).TRADE_RATE.agg(['min','max'])
df2['OPEN']=df1.groupby([df1.TRADE_ENTRY_DATE,df1.Date_Time.dt.hour,df1.Date_Time.dt.minute]).TRADE_RATE.first()
df2['Close']=df1.groupby([df1.TRADE_ENTRY_DATE,df1.Date_Time.dt.hour,df1.Date_Time.dt.minute]).TRADE_RATE.nth(-1)
df2.columns=['LOW','HIGH','OPEN','CLOSE']
df2.head()

LOW   HIGH  OPEN  CLOSE
TRADE_ENTRY_DATE Date_Time Date_Time ATRL                           
2009-04-01       9         30        ATRL  82.00  83.50  82.0  82.90
                           31        ATRL  82.52  83.30  82.9  83.30
                           32        ATRL  83.25  85.00  83.3  85.00
                           33        ATRL  84.98  85.49  85.0  85.40
                           34        ATRL  85.40  87.00  85.4  86.85

In [13]:
df2.shape

(343, 4)

In [ ]:
df1=df[:]
df2=df1.groupby([df1.TRADE_ENTRY_DATE,df1.Date_Time.dt.hour,df1.Date_Time.dt.minute,df1.ATRL]).TRADE_RATE.agg(['min','max'])
df2['TRADE_QTY']=df1.groupby([df1.TRADE_ENTRY_DATE,df1.Date_Time.dt.hour,df1.Date_Time.dt.minute]).TRADE_RATE.first()
df2['TRADE_RATE']=df1.groupby([df1.TRADE_ENTRY_DATE,df1.Date_Time.dt.hour,df1.Date_Time.dt.minute]).TRADE_QTY.first()
df2.columns=['LOW','HIGH','TRADE_RATE','TRADE_QTY']

In [ ]:
df2.shape

In [30]:
df1=pd.read_csv('../Modified_Data/Modified_bahl.csv',parse_dates=True,index_col='Date_Time',infer_datetime_format=True)
df1.drop(['Unnamed: 0','KATS_TIME'],axis=1,inplace=True)
df1.reset_index(inplace=True)
df1.columns=['BAHL_DT','BAHL_TRADE_DATE','BAHL','BAHL_TRADE_QTY','BAHL_TRADE_RATE']
df2=pd.concat([df,df1],axis=1)

In [31]:
df2

,Date_Time,TRADE_ENTRY_DATE,ATRL,TRADE_QTY,TRADE_RATE,BAHL_DT,BAHL_TRADE_DATE,BAHL,BAHL_TRADE_QTY,BAHL_TRADE_RATE
0,2009-04-01 09:30:01.098700,2009-04-01,ATRL,100,82.00,2009-04-01 09:33:09.007600,2009-04-01,BAHL,200.0,26.92
1,2009-04-01 09:30:01.098800,2009-04-01,ATRL,1000,82.00,2009-04-01 09:38:30.004700,2009-04-01,BAHL,1000.0,27.20
2,2009-04-01 09:30:01.098900,2009-04-01,ATRL,100,82.00,2009-04-01 09:38:39.000200,2009-04-01,BAHL,5000.0,27.00
3,2009-04-01 09:30:01.099000,2009-04-01,ATRL,300,82.00,2009-04-01 09:40:03.004500,2009-04-01,BAHL,5000.0,27.00
4,2009-04-01 09:30:01.099100,2009-04-01,ATRL,3700,82.00,2009-04-01 09:43:34.002400,2009-04-01,BAHL,2000.0,27.00
...,...,...,...,...,...,...,...,...,...,...
4302036,2019-04-15 15:28:45.005000,2019-04-15,ATRL,100,109.70,NaT,NaN,NaN,NaN,NaN
4302037,2019-04-15 15:28:51.000700,2019-04-15,ATRL,100,109.70,NaT,NaN,NaN,NaN,NaN
4302038,2019-04-15 15:28:51.002100,2019-04-15,ATRL,200,109.55,NaT,NaN,NaN,NaN,NaN
4302039,2019-04-15 15:29:15.005000,2019-04-15,ATRL,200,109.56,NaT,NaN,NaN,NaN,NaN


In [27]:
df2.BAHL.fillna('BAHL',inplace=True)
df2.drop('BAHL_DT',inplace=True)
df2.BAHL_TRADE_QTY.fillna(0,inplace=True)
df2.BAHL_TRADE_RATE.fillna(0,inplace=True)
df2.head()

/home/rufus/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


KeyError: "['TRADE_ENTRY_DATE'] not found in axis"

In [ ]:
df2.shape

In [4]:
Names=['bahl','EPCL','FCCL','LUCK','MCB','MLCF','NETSOL','NML',
      'OGDC','PPL','PSO','SEARL','TRG']
df=pd.read_csv('../Modified_Data/Modified_ATRL.csv',parse_dates=True,index_col='Date_Time',infer_datetime_format=True)
df.drop(['Unnamed: 0','KATS_TIME'],axis=1,inplace=True)
df.reset_index(inplace=True)
for i in range(len(Names)):
    df1=pd.read_csv('../Modified_Data/Modified_'+Names[i]+'.csv',parse_dates=True,index_col='Date_Time',infer_datetime_format=True)
    df1.drop(['Unnamed: 0','KATS_TIME'],axis=1,inplace=True)
    df1.reset_index(inplace=True)
    df1.columns=[Names[i]+'_DT',Names[i]+'_TRADE_DATE',Names[i],Names[i]+'_TRADE_QTY',Names[i]+'_TRADE_RATE']
    df=pd.concat([df,df1],axis=1)


In [5]:
df.drop(['bahl_DT','EPCL_DT','FCCL_DT','LUCK_DT','MCB_DT','MLCF_DT','NETSOL_DT','NML_DT','OGDC_DT','PPL_DT',
        'OGDC_DT','PPL_DT','PSO_DT','SEARL_DT','TRG_DT'],axis=1,inplace=True)
df.drop(['bahl_TRADE_DATE','EPCL_TRADE_DATE','FCCL_TRADE_DATE','LUCK_TRADE_DATE','MCB_TRADE_DATE','MLCF_TRADE_DATE',
         'NETSOL_TRADE_DATE','NML_TRADE_DATE','OGDC_TRADE_DATE','PPL_TRADE_DATE',
        'OGDC_TRADE_DATE','PPL_TRADE_DATE','PSO_TRADE_DATE','SEARL_TRADE_DATE','TRG_TRADE_DATE'],axis=1,inplace=True)

In [17]:
df.bahl.fillna('BAHL',inplace=True),df.EPCL.fillna('EPCL',inplace=True),df.FCCL.fillna('FCCL',inplace=True),
df.LUCK.fillna('LUCK',inplace=True),df.MCB.fillna('MCB',inplace=True),
df.MLCF.fillna('MLCF',inplace=True),df.NETSOL.fillna('NETSOL',inplace=True),df.NML.fillna('NML',inplace=True),
df.OGDC.fillna('OGDC',inplace=True),df.PPL.fillna('PPL',inplace=True),
df.PSO.fillna('PSO',inplace=True),df.SEARL.fillna('SEARL',inplace=True),df.TRG.fillna('TRG',inplace=True)
df.fillna(0,inplace=True)

In [23]:
df

,Date_Time,TRADE_ENTRY_DATE,SYMBOL_CODE,TRADE_QTY,TRADE_RATE,bahl,bahl_TRADE_QTY,bahl_TRADE_RATE,EPCL,EPCL_TRADE_QTY,...,PPL_TRADE_RATE,PSO,PSO_TRADE_QTY,PSO_TRADE_RATE,SEARL,SEARL_TRADE_QTY,SEARL_TRADE_RATE,TRG,TRG_TRADE_QTY,TRG_TRADE_RATE
0,2009-04-01 09:30:01.098700,2009-04-01,ATRL,100,82.00,BAHL,200.0,26.92,EPCL,1000.0,...,175.0,PSO,1600.0,199.0,SEARL,2000.0,43.10,TRG,500.0,1.40
1,2009-04-01 09:30:01.098800,2009-04-01,ATRL,1000,82.00,BAHL,1000.0,27.20,EPCL,500.0,...,175.0,PSO,100.0,199.0,SEARL,500.0,45.50,TRG,500.0,1.39
2,2009-04-01 09:30:01.098900,2009-04-01,ATRL,100,82.00,BAHL,5000.0,27.00,EPCL,500.0,...,175.0,PSO,100.0,199.0,SEARL,500.0,46.00,TRG,500.0,1.40
3,2009-04-01 09:30:01.099000,2009-04-01,ATRL,300,82.00,BAHL,5000.0,27.00,EPCL,500.0,...,175.0,PSO,100.0,199.0,SEARL,500.0,45.01,TRG,500.0,1.39
4,2009-04-01 09:30:01.099100,2009-04-01,ATRL,3700,82.00,BAHL,2000.0,27.00,EPCL,500.0,...,175.0,PSO,500.0,199.0,SEARL,500.0,46.00,TRG,1000.0,1.39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4302036,2019-04-15 15:28:45.005000,2019-04-15,ATRL,100,109.70,bahl,0.0,0.00,EPCL,0.0,...,0.0,PSO,0.0,0.0,SEARL,0.0,0.00,TRG,0.0,0.00
4302037,2019-04-15 15:28:51.000700,2019-04-15,ATRL,100,109.70,bahl,0.0,0.00,EPCL,0.0,...,0.0,PSO,0.0,0.0,SEARL,0.0,0.00,TRG,0.0,0.00
4302038,2019-04-15 15:28:51.002100,2019-04-15,ATRL,200,109.55,bahl,0.0,0.00,EPCL,0.0,...,0.0,PSO,0.0,0.0,SEARL,0.0,0.00,TRG,0.0,0.00
4302039,2019-04-15 15:29:15.005000,2019-04-15,ATRL,200,109.56,bahl,0.0,0.00,EPCL,0.0,...,0.0,PSO,0.0,0.0,SEARL,0.0,0.00,TRG,0.0,0.00


In [ ]:
df1=df[:]
df2=df1.groupby([df1.TRADE_ENTRY_DATE,df1.Date_Time.hour,df1.Date_Time.minute,df1.SYMBOL_CODE]).TRADE_RATE.agg(['min','max'])
df2['TRADE_QTY']=df1.groupby([df1.TRADE_ENTRY_DATE,df1.Date_Time.hour,df1.Date_Time.minute]).TRADE_RATE.first()
df2['TRADE_RATE']=df1.groupby([df1.TRADE_ENTRY_DATE,df1.Date_Time.hour,df1.Date_Time.minute]).TRADE_QTY.first()
df2.columns=['LOW','HIGH','TRADE_RATE','TRADE_QTY']

In [ ]:
df1=df[:]
df2=df1.groupby([df1.TRADE_ENTRY_DATE,df1.Date_Time.dt.hour,df1.Date_Time.dt.minute,df1.SYMBOL_CODE]).TRADE_RATE.agg(['min','max'])
df2['TRADE_QTY']=df1.groupby([df1.TRADE_ENTRY_DATE,df1.Date_Time.dt.hour,df1.Date_Time.dt.minute]).TRADE_RATE.first()
df2['TRADE_RATE']=df1.groupby([df1.TRADE_ENTRY_DATE,df1.Date_Time.dt.hour,df1.Date_Time.dt.minute]).TRADE_QTY.first()
df2.columns=['LOW','HIGH','TRADE_RATE','TRADE_QTY']

In [ ]:
df2.shape

# Installing Gradient Boosted Tree model

In [ ]:
!pip install XGBoost

In [ ]:
data=df

In [ ]:
from sklearn import model_selection,tree
import xgboost as xgb
from sklearn import metrics

In [ ]:
df['date'] = df.index
df['hour'] = df['date'].dt.hour
df['dayofweek'] = df['date'].dt.dayofweek
df['quarter'] = df['date'].dt.quarter
df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year
df['dayofyear'] = df['date'].dt.dayofyear
df['dayofmonth'] = df['date'].dt.day
df['weekofyear'] = df['date'].dt.weekofyear
    
X = df[['hour','dayofweek','quarter','month','year',
           'dayofyear','dayofmonth','weekofyear','Open','High','Low','Volume']]

In [ ]:
X.head()

In [ ]:
X.Open=X.Open.astype('float')
X.High=X.High.astype('float')
X.Low=X.Low.astype('float')
X.Volume=X.Volume.astype('float')

In [ ]:
y=data[['Close']]

In [ ]:
y=y.astype('float')
X

In [ ]:
X_train,X_test,y_train,y_test=model_selection.train_test_split(X,y,test_size=0.2)

# Training model

In [ ]:
reg = xgb.XGBRegressor(n_estimators=1000)
reg.fit(X_train, y_train)

In [ ]:
yhat=reg.predict(X_test)

In [ ]:
#sns.lineplot(data=yhat)
sns.lineplot(data=y_test)

In [ ]:
sns.lineplot(data=yhat)


In [ ]:
metrics.mean_squared_error(y_test,yhat)

In [ ]:
yhat=reg.predict(X_train)

In [ ]:
sns.lineplot(data=yhat)

In [ ]:
sns.lineplot(data=y_test)